In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta
import os
import sys

# ==================================
# ========== GitHubCollector 클래스 ==========
# ==================================

class GitHubCollector:
    def __init__(self, token):
        self.token = token
        self.headers = {
            "Authorization": f"Bearer {token}",
            "Accept": "application/vnd.github+json"
        }
        self.base_url = "https://api.github.com"

    def check_rate_limit(self):
        """API 사용량 확인 및 검색 Rate Limit 반환"""
        url = f"{self.base_url}/rate_limit"
        try:
            response = requests.get(url, headers=self.headers, timeout=10)
            response.raise_for_status()
            data = response.json()
            rate_info = data['resources']['search']

            remaining = rate_info['remaining']
            limit = rate_info['limit']
            reset_time = datetime.fromtimestamp(rate_info['reset'])

            # sys.stdout.write를 사용하여 출력 버퍼를 관리하고 현재 상태를 표시
            sys.stdout.write(f"⏱️ 남은 요청: {remaining}/{limit} (리셋: {reset_time.strftime('%H:%M:%S')})")
            return remaining, reset_time

        except requests.exceptions.RequestException as e:
            sys.stdout.write(f"❌ Rate Limit 체크 중 에러 발생: {e}")
            return 0, datetime.now() + timedelta(minutes=1)

    def search_repos(self, keyword, start_date, end_date, per_page=100, max_pages=10):
        """단일 날짜로 레포지토리 검색 (최대 1000개)"""
        url = f"{self.base_url}/search/repositories"
        date_query = f"created:{start_date}..{end_date}"
        query = f"{keyword} {date_query}"

        params = {
            "q": query,
            "per_page": per_page,
            "sort": "stars",
            "order": "desc",
            "page": 1
        }

        all_repos = []
        retry_count = 0
        max_retries = 5

        while params['page'] <= max_pages:
            try:
                response = requests.get(url, headers=self.headers, params=params, timeout=15)

                if response.status_code == 200:
                    data = response.json()
                    repos = data.get('items', [])

                    if not repos:
                        break

                    all_repos.extend(repos)

                    if len(all_repos) >= per_page * max_pages:
                        break

                    params['page'] += 1
                    retry_count = 0
                    time.sleep(0.5)

                elif response.status_code == 403 and 'rate limit' in response.text:
                    sys.stdout.write(f"\n⚠️ Rate Limit 초과 (403). 재시도 대기 중...")
                    reset_timestamp = int(response.headers.get('X-RateLimit-Reset', time.time() + 60))
                    wait_time = max(10, reset_timestamp - int(time.time()) + 5)
                    sys.stdout.write(f" ({wait_time:.0f}초)\n")
                    time.sleep(wait_time)
                    retry_count += 1
                    if retry_count > max_retries:
                        sys.stdout.write("❌ 최대 재시도 횟수 초과. 검색 중단.\n")
                        break

                else:
                    sys.stdout.write(f"\n❌ API 에러: {response.status_code} - {response.text[:100]}...\n")
                    break

            except requests.exceptions.RequestException as e:
                sys.stdout.write(f"\n⚠️ 요청 에러 발생: {e}. 10초 후 재시도.")
                time.sleep(10)
                retry_count += 1
                if retry_count > max_retries:
                    sys.stdout.write("❌ 최대 재시도 횟수 초과. 검색 중단.\n")
                    break

        return all_repos

    def _chunk_list(self, lst, n):
        """리스트를 n개 이하의 청크로 나눔"""
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

    def collect_year(self, keywords, year):
        """
        1년치 데이터 수집 (일별) - 2025년은 11월 30일까지 제한
        """
        all_repos = []
        # 키워드 청크 분할 (OR 연산자 5개 제한 회피)
        keyword_chunks = list(self._chunk_list(keywords, 5))

        # 📌 날짜 범위 설정
        start_date = datetime(year, 1, 1).date()
        end_date = datetime(year, 12, 31).date()

        # 2025년은 현재 날짜(2025년 11월 17일)를 고려하여 11월 30일까지만 수집하도록 설정
        if year == 2025:
            end_date = datetime(2025, 11, 30).date()

        total_days = (end_date - start_date).days + 1
        current_day = 0

        print("\n" + "="*70)
        print(f"📅 {year}년 데이터 수집 시작")
        print(f"🔍 키워드 청크 수: {len(keyword_chunks)}개")
        print(f"📆 기간: {start_date.strftime('%Y-%m-%d')} ~ {end_date.strftime('%Y-%m-%d')} ({total_days}일)")
        print("="*70)

        current_date = start_date
        while current_date <= end_date:
            current_day += 1
            date_str = current_date.strftime("%Y-%m-%d")
            daily_repos = []

            sys.stdout.write(f"\n[{current_day}/{total_days}] 🔍 {date_str} ")

            # Rate Limit 체크
            remaining, reset_time = self.check_rate_limit()
            if remaining < 5 * len(keyword_chunks):
                wait_seconds = max(10, (reset_time - datetime.now()).total_seconds() + 5)
                sys.stdout.write(f"\n⏸️ API 제한 임박 ({remaining}개 남음). {wait_seconds:.0f}초 대기 중...\n")
                time.sleep(wait_seconds)
                sys.stdout.write(f"재개: [{current_day}/{total_days}] 🔍 {date_str} ")

            # 키워드 청크별 반복 검색
            for i, chunk in enumerate(keyword_chunks):
                keyword_string = " OR ".join(chunk)

                repos = self.search_repos(keyword_string, date_str, date_str)
                daily_repos.extend(repos)

                if len(keyword_chunks) > 1 and i < len(keyword_chunks) - 1:
                    sys.stdout.write(f" (청크 {i+1}/{len(keyword_chunks)} 완료. 1초 대기)")
                    time.sleep(1)

            all_repos.extend(daily_repos)

            sys.stdout.write(f"→ {len(daily_repos)}개 수집 (누적: {len(all_repos)}개)\n")

            current_date += timedelta(days=1)

        print("\n" + "="*70)
        print(f"🎉 {year}년 수집 완료: 총 {len(all_repos)}개 (중복 포함)")
        print("="*70)

        return all_repos

    def repos_to_dataframe(self, repos):
        """레포지토리 리스트를 DataFrame으로 변환 (중복 제거 로직 포함)"""
        # ID 기준으로 중복 레포지토리를 제거 (중복 처리)
        unique_repos = {repo['id']: repo for repo in repos}.values()

        data = []
        for repo in unique_repos:
            owner = repo.get('owner', {})
            license_info = repo.get('license')

            data.append({
                'id': repo['id'],
                'name': repo['name'],
                'full_name': repo['full_name'],
                'owner': owner.get('login', 'N/A'),
                'owner_type': owner.get('type', 'N/A'),
                'description': repo.get('description', ''),
                'language': repo.get('language', 'Unknown'),
                'stars': repo.get('stargazers_count', 0),
                'forks': repo.get('forks_count', 0),
                'watchers': repo.get('watchers_count', 0),
                'open_issues': repo.get('open_issues_count', 0),
                'created_at': repo.get('created_at'),
                'updated_at': repo.get('updated_at'),
                'pushed_at': repo.get('pushed_at'),
                'size': repo.get('size', 0),
                'url': repo.get('html_url'),
                'topics': '|'.join(repo.get('topics', [])),
                'license': license_info.get('name') if license_info else 'No License',
                'default_branch': repo.get('default_branch', 'main'),
                'has_wiki': repo.get('has_wiki', False),
                'has_pages': repo.get('has_pages', False),
                'archived': repo.get('archived', False)
            })

        df = pd.DataFrame(data)
        print(f"\n📊 DataFrame 생성: {len(df)}개 레코드 (고유 레포)")
        return df

    def save_to_csv(self, df, filename):
        """CSV로 저장"""
        if not os.path.exists('data'):
            os.makedirs('data')

        filepath = os.path.join('data', filename)
        df.to_csv(filepath, index=False, encoding='utf-8-sig')

        file_size = os.path.getsize(filepath) / 1024 / 1024

        print(f"\n💾 저장 완료!")
        print(f"📁 경로: {os.path.abspath(filepath)}")
        print(f"📊 크기: {file_size:.2f} MB")
        print(f"📋 레코드: {len(df)}개")

        return filepath


# ==================================
# ========== 실행 코드 (MAIN) - 2021년 ~ 2025년 ==========
# ==================================

# 🔑 GitHub Token 입력
try:
    GITHUB_TOKEN = input("GitHub Token을 입력하세요: ")
except EOFError:
    print("\n⚠️ 입력이 감지되지 않아 프로그램을 종료합니다. 터미널 환경에서 실행해주세요.")
    sys.exit()

if not GITHUB_TOKEN:
    print("❌ 토큰이 입력되지 않아 프로그램을 종료합니다.")
    sys.exit()

try:
    collector = GitHubCollector(GITHUB_TOKEN)
except Exception as e:
    print(f"❌ Collector 초기화 중 오류 발생: {e}")
    sys.exit()

# 전체 AI 키워드 (12개)
AI_KEYWORDS = [
    "chatgpt",
    "gpt-4",
    "openai-api",
    "AI-agent",
    "langchain",
    "autonomous-agent",
    "github-copilot",
    "machine-learning",
    "artificial-intelligence",
    "deep-learning",
    "neural-network",
    "llm"
]

# Rate Limit 확인
collector.check_rate_limit()

# 📌 수집 연도를 2021년부터 2025년까지 설정
years = [2021, 2022, 2023, 2024, 2025]

for year in years:
    print(f"\n\n{'='*70}")
    print(f"🎯 {year}년 데이터 수집 시작")
    print(f"{'='*70}")

    # 연도별 데이터 수집
    repos = collector.collect_year(AI_KEYWORDS, year)

    df = collector.repos_to_dataframe(repos)

    # 기본 통계
    print(f"\n📊 {year}년 통계:")
    print(f"  총 레코드: {len(df)}개")
    print(f"  고유 레포: {df['id'].nunique()}개")
    print(f"\n  언어별 Top 5:")
    print(df['language'].value_counts().head())

    filename = f'github_{year}.csv'

    # 2025년은 부분 수집임을 파일명에 명시
    if year == 2025:
        filename = f'github_{year}_part.csv'

    collector.save_to_csv(df, filename)

    print(f"\n✅ {year}년 수집 완료!")

    # 마지막 연도가 아니라면 다음 연도 수집 전 대기
    if year != years[-1]:
        print("\n⏸️ 다음 연도 수집 전 10초 대기...")
        time.sleep(10)

print("\n" + "="*70)
print("🎉 전체 수집 완료!")
print("="*70)
print("\n📁 생성된 파일:")
for year in years:
    filename = f'github_{year}.csv'
    if year == 2025:
        filename = f'github_{year}_part.csv'
    print(f"  - data/{filename}")

⏱️ 남은 요청: 30/30 (리셋: 12:41:11)

🎯 2025년 데이터 수집 시작

📅 2025년 데이터 수집 시작
🔍 키워드 청크 수: 3개
📆 기간: 2025-01-01 ~ 2025-11-30 (334일)

[1/334] 🔍 2025-01-01 ⏱️ 남은 요청: 30/30 (리셋: 12:41:11) (청크 1/3 완료. 1초 대기) (청크 2/3 완료. 1초 대기)→ 1056개 수집 (누적: 1056개)

[2/334] 🔍 2025-01-02 ⏱️ 남은 요청: 14/30 (리셋: 12:41:14)
⏸️ API 제한 임박 (14개 남음). 16초 대기 중...
재개: [2/334] 🔍 2025-01-02  (청크 1/3 완료. 1초 대기) (청크 2/3 완료. 1초 대기)→ 1538개 수집 (누적: 2594개)

[3/334] 🔍 2025-01-03 ⏱️ 남은 요청: 27/30 (리셋: 12:43:25) (청크 1/3 완료. 1초 대기) (청크 2/3 완료. 1초 대기)→ 1450개 수집 (누적: 4044개)

[4/334] 🔍 2025-01-04 ⏱️ 남은 요청: 27/30 (리셋: 12:44:29) (청크 1/3 완료. 1초 대기) (청크 2/3 완료. 1초 대기)→ 1348개 수집 (누적: 5392개)

[5/334] 🔍 2025-01-05 ⏱️ 남은 요청: 28/30 (리셋: 12:45:31) (청크 1/3 완료. 1초 대기) (청크 2/3 완료. 1초 대기)→ 1428개 수집 (누적: 6820개)

[6/334] 🔍 2025-01-06 ⏱️ 남은 요청: 27/30 (리셋: 12:46:34) (청크 1/3 완료. 1초 대기) (청크 2/3 완료. 1초 대기)→ 1567개 수집 (누적: 8387개)

[7/334] 🔍 2025-01-07 ⏱️ 남은 요청: 25/30 (리셋: 12:47:35) (청크 1/3 완료. 1초 대기) (청크 2/3 완료. 1초 대기)→ 1690개 수집 (누적: 10077개)

[8/334] 🔍 2025-01-08 ⏱️ 남